# Processing Column Data

We will explore the functions available under pyspark.sql.functions to derive new values from existing column values within a dataframe.
* [Pre-defined Functions](#first)
* [Create dummy dataframe](#sec)
* [Categories of functions](#third)
* [Special fucntions - col and lit](#fourth)
* [String manipulation functions - 1](#fifth)
* [String manipulation functions - 2](#sixth)
* [Date and Time overview](#seventh)
* [Date and Time arithmetic](#eigth)
* [Date and Time - trunc and date_trunc](#ninth)
* [Date and Time - extracting information](#tenth)
* [Dealing with UNIX timestamp](#eleventh)
* [Dealing with Nulls](#twelth)
* [Conclusion](#thirteen)

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
spark = SparkSession.builder.appName('SparkFunctions').getOrCreate()

### Pre-defined Functions<a id="first"></a>

We typically process data in the columns using functions in `pyspark.sl.functions`. Let us understand details about these functions in detail as part of this module.

* Let us recap about functions or API to proces DataFrames.
    * Projection - `select` or `withColumn` or `drop` or `selectExpr`
    * Filtering - `filter` or `where`
    * Grouping data by key and perform aggregations - `groupBy`
    * Sorting data - `sort` or `orderBy`
* We can pass column names or literals or expressions to all the dataframe APIs.
* Expressions include arithmetic operations, transformations using functions from `pyspark.sql.functions`.
* There are approximately 300 functions under `pyspark.sql.functions`.
* There are some important functions related to String Manipulation, Date Manipulation etc

In [3]:
orders = spark.read.csv('../data/orders.csv', 
        schema='order_id INT, order_date STRING, order_customer_id INT, order_status STRING')

In [4]:
orders.show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

In [5]:
# Schema
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



#### date_format

In [6]:
help(date_format)

Help on function date_format in module pyspark.sql.functions:

date_format(date, format)
    Converts a date/timestamp/string to a value of string in the format specified by the date
    format given by the second argument.
    
    A pattern could be for instance `dd.MM.yyyy` and could return a string like '18.03.1993'. All
    pattern letters of `datetime pattern`_. can be used.
    
    .. _datetime pattern: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
    .. note:: Use when ever possible specialized functions like `year`. These benefit from a
        specialized implementation.
    
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(date_format('dt', 'MM/dd/yyy').alias('date')).collect()
    [Row(date='04/08/2015')]
    
    .. versionadded:: 1.5



In [17]:
# dateformat with select and alias
orders.select('*',
             date_format('order_date', 'yyyyMM').alias('order_month')).show(5)

+--------+--------------------+-----------------+---------------+-----------+
|order_id|          order_date|order_customer_id|   order_status|order_month|
+--------+--------------------+-----------------+---------------+-----------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|     201307|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|     201307|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|     201307|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|     201307|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|     201307|
+--------+--------------------+-----------------+---------------+-----------+
only showing top 5 rows



In [19]:
# dateformat with withColumn
orders.withColumn('order_month', date_format('order_date', 'yyyyMM')).show(5)

+--------+--------------------+-----------------+---------------+-----------+
|order_id|          order_date|order_customer_id|   order_status|order_month|
+--------+--------------------+-----------------+---------------+-----------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|     201307|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|     201307|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|     201307|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|     201307|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|     201307|
+--------+--------------------+-----------------+---------------+-----------+
only showing top 5 rows



In [40]:
datetimes = [("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                ]

In [42]:
datetimesDF = spark.createDataFrame(datetimes, schema="date STRING, time STRING")
datetimesDF.show(truncate=False)

+----------+-----------------------+
|date      |time                   |
+----------+-----------------------+
|2014-02-28|2014-02-28 10:00:00.123|
|2016-02-29|2016-02-29 08:08:08.999|
|2017-10-31|2017-12-31 11:59:59.123|
|2019-11-30|2019-08-31 00:00:00.000|
+----------+-----------------------+



In [44]:
datetimesDF. \
withColumn("date_yd", date_format("date", "yyyyDDD").cast("int")). \
withColumn("time_yd", date_format("time", "yyyyDDD").cast("int")). \
show()

+----------+--------------------+-------+-------+
|      date|                time|date_yd|time_yd|
+----------+--------------------+-------+-------+
|2014-02-28|2014-02-28 10:00:...|2014059|2014059|
|2016-02-29|2016-02-29 08:08:...|2016060|2016060|
|2017-10-31|2017-12-31 11:59:...|2017304|2017365|
|2019-11-30|2019-08-31 00:00:...|2019334|2019243|
+----------+--------------------+-------+-------+



In [48]:
datetimesDF. \
withColumn("date_yd", date_format("date", "MMMM d, yyyy")). \
withColumn("date_name_abbr", date_format("date", "EE")). \
withColumn("date_name_full", date_format("date", "EEEE")). \
show()

+----------+--------------------+-----------------+--------------+--------------+
|      date|                time|          date_yd|date_name_abbr|date_name_full|
+----------+--------------------+-----------------+--------------+--------------+
|2014-02-28|2014-02-28 10:00:...|February 28, 2014|           Fri|        Friday|
|2016-02-29|2016-02-29 08:08:...|February 29, 2016|           Mon|        Monday|
|2017-10-31|2017-12-31 11:59:...| October 31, 2017|           Tue|       Tuesday|
|2019-11-30|2019-08-31 00:00:...|November 30, 2019|           Sat|      Saturday|
+----------+--------------------+-----------------+--------------+--------------+



#### filter

In [20]:
help(filter)

Help on class filter in module builtins:

class filter(object)
 |  filter(function or None, iterable) --> filter object
 |  
 |  Return an iterator yielding those items of iterable for which function(item)
 |  is true. If function is None, return the items that are true.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



#### groupBy

In [26]:
help(orders.groupBy)

Help on method groupBy in module pyspark.sql.dataframe:

groupBy(*cols) method of pyspark.sql.dataframe.DataFrame instance
    Groups the :class:`DataFrame` using the specified columns,
    so we can run aggregation on them. See :class:`GroupedData`
    for all the available aggregate functions.
    
    :func:`groupby` is an alias for :func:`groupBy`.
    
    :param cols: list of columns to group by.
        Each element should be a column name (string) or an expression (:class:`Column`).
    
    >>> df.groupBy().avg().collect()
    [Row(avg(age)=3.5)]
    >>> sorted(df.groupBy('name').agg({'age': 'mean'}).collect())
    [Row(name='Alice', avg(age)=2.0), Row(name='Bob', avg(age)=5.0)]
    >>> sorted(df.groupBy(df.name).avg().collect())
    [Row(name='Alice', avg(age)=2.0), Row(name='Bob', avg(age)=5.0)]
    >>> sorted(df.groupBy(['name', df.age]).count().collect())
    [Row(name='Alice', age=2, count=1), Row(name='Bob', age=5, count=1)]
    
    .. versionadded:: 1.3



In [22]:
# function as a part of where or filter
orders.filter(date_format('order_date', 'yyyyMM') == 201401).show(5)

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|   25876|2014-01-01 00:00:...|             3414|PENDING_PAYMENT|
|   25877|2014-01-01 00:00:...|             5549|PENDING_PAYMENT|
|   25878|2014-01-01 00:00:...|             9084|        PENDING|
|   25879|2014-01-01 00:00:...|             5118|        PENDING|
|   25880|2014-01-01 00:00:...|            10146|       CANCELED|
+--------+--------------------+-----------------+---------------+
only showing top 5 rows



In [24]:
# function as a part of groupBy
orders.groupBy(date_format('order_date', 'yyyyMM').alias('order_month')).count().show(5)

+-----------+-----+
|order_month|count|
+-----------+-----+
|     201401| 5908|
|     201405| 5467|
|     201312| 5892|
|     201310| 5335|
|     201311| 6381|
+-----------+-----+
only showing top 5 rows



### Create dummy dataframe<a id="sec"></a>

In [27]:
l = [('X', )]

In [31]:
df = spark.createDataFrame(l, 'dummy STRING')

In [32]:
df.printSchema()

root
 |-- dummy: string (nullable = true)



In [33]:
df.show()

+-----+
|dummy|
+-----+
|    X|
+-----+



Once dataframe is created, we can use to understand how to use functions. for examples, to get current date, we can run `df.select(current_date()).show()`

It is similar to Oracle Query `SELECT sysdate FROM dual`

In [35]:
df.select(current_date().alias('current_date')).show()

+------------+
|current_date|
+------------+
|  2022-01-27|
+------------+



Here is the example of creating dataframe using collection of employees. we will be using this dataframe to explore all the important functions to process column data in detail.

In [45]:
employees = [(1, "Scott", 'Tiger', 1000.0, "USA", "+1 123 456 7890", "123 45 6789"),
            (2, "Henry", 'Ford', 750.0, "UK", "+1 123 456 7890", "123 45 6789")
            ]

In [46]:
employeeDF = spark.createDataFrame(employees, schema="""id INT, f_name STRING, l_name STRING, salary FLOAT, nationality STRING, ph_no STRING, ssn STRING""")

In [39]:
employeeDF.printSchema()

root
 |-- id: integer (nullable = true)
 |-- f_name: string (nullable = true)
 |-- l_name: string (nullable = true)
 |-- salary: float (nullable = true)
 |-- nationality: string (nullable = true)
 |-- ph_no: string (nullable = true)
 |-- ssn: string (nullable = true)



In [40]:
employeeDF.show(truncate=False)

+---+------+------+------+-----------+---------------+-----------+
|id |f_name|l_name|salary|nationality|ph_no          |ssn        |
+---+------+------+------+-----------+---------------+-----------+
|1  |Scott |Tiger |1000.0|USA        |+1 123 456 7890|123 45 6789|
|1  |Henry |Ford  |750.0 |UK         |+1 123 456 7890|123 45 6789|
+---+------+------+------+-----------+---------------+-----------+



### Categories of functions<a id="third"></a>

There are approx 300 functions under `pyspark.sql.functions`. At a higher level they canbe grouped into a few categories.

* String Manipulation
    * Case conversion - `lower`, `upper`
    * Getting length - `length`
    * Extracting substrings - `substring`, `split`
    * Trimming - `trim`, `ltrim`, `rtrim`
    * Padding - `lpad`, `rpad`
    * Concatenating string - `concat`, `concat_ws`
* Date Manipulation
    * Getting current date and time - `current_date`, `current_timestamp`
    * Date arithmetic - `date_add`, `date_sub`, `datediff`, `months_between`, `add_months`, `next_day`
    * Begining and Ending Date or Time - `last_day`, `trunc`, `date_trunc`
    * Formatting Date - `date_format`
    * Extracting Information - `dayofyear`, `dayofmonth`, `dayofweek`, `year`, `month`
* Aggregate Functions
    * `count`, `countDistinct`
    * `sum`, `avg`
    * `min`, `max`
* Other Functions - We will explore depending on the use cases.
    * `CASE` and `WHEN`
    * `CAST` for type casting
    * Functions to manage special types such as `ARRAY`, `MAP`, `STRUCT` type columns
    * Many others 

### Special functions - col and lit<a id="fourth"></a>

* For dataframe APIs such as `select`, `groupBy`, `orderBy` etc we can pass column names as strings.

In [42]:
employeeDF.select("f_name", "l_name").show()

+------+------+
|f_name|l_name|
+------+------+
| Scott| Tiger|
| Henry|  Ford|
+------+------+



In [44]:
employeeDF.groupBy("nationality").count().show()

+-----------+-----+
|nationality|count|
+-----------+-----+
|        USA|    1|
|         UK|    1|
+-----------+-----+



In [52]:
employeeDF.orderBy(desc("id")).show()

+---+------+------+------+-----------+---------------+-----------+
| id|f_name|l_name|salary|nationality|          ph_no|        ssn|
+---+------+------+------+-----------+---------------+-----------+
|  2| Henry|  Ford| 750.0|         UK|+1 123 456 7890|123 45 6789|
|  1| Scott| Tiger|1000.0|        USA|+1 123 456 7890|123 45 6789|
+---+------+------+------+-----------+---------------+-----------+



* If there are no transformations on any column in any function then we should be able to pass all column names as strings.
* If not, we need to pass all columns as type column by col function.
* If we need to apply any transformation using functions then passing column names as strings to some of the functions will not suffice. We have to pass them as column type.

In [57]:
employeeDF.select(upper(col("f_name")), upper(col("l_name"))).show()

+-------------+-------------+
|upper(f_name)|upper(l_name)|
+-------------+-------------+
|        SCOTT|        TIGER|
|        HENRY|         FORD|
+-------------+-------------+



In [59]:
# Alternate Approach 1
employeeDF.select(upper(employeeDF["f_name"]), upper(employeeDF["l_name"])).show()

+-------------+-------------+
|upper(f_name)|upper(l_name)|
+-------------+-------------+
|        SCOTT|        TIGER|
|        HENRY|         FORD|
+-------------+-------------+



In [60]:
# Alternate Approach 2
employeeDF.select(upper(employeeDF["f_name"]), upper(employeeDF["l_name"])).show()

+-------------+-------------+
|upper(f_name)|upper(l_name)|
+-------------+-------------+
|        SCOTT|        TIGER|
|        HENRY|         FORD|
+-------------+-------------+



* Also, if we want to use functions such as `alias`, `desc` etc on columns then we have to pass the column names as column type(not as strings)

In [61]:
employeeDF.orderBy("id".desc()).show()

AttributeError: 'str' object has no attribute 'desc'

In [62]:
employeeDF.orderBy(col("id").desc()).show()

+---+------+------+------+-----------+---------------+-----------+
| id|f_name|l_name|salary|nationality|          ph_no|        ssn|
+---+------+------+------+-----------+---------------+-----------+
|  2| Henry|  Ford| 750.0|         UK|+1 123 456 7890|123 45 6789|
|  1| Scott| Tiger|1000.0|        USA|+1 123 456 7890|123 45 6789|
+---+------+------+------+-----------+---------------+-----------+



* Sometimes, we want to add a literal to the column values. For e.g., we might want to concatenate first_name and last_name seperated by comma and space in between.

In [63]:
employeeDF.select(concat(col("f_name"), ", ", col("l_name"))).show()

AnalysisException: cannot resolve '`, `' given input columns: [f_name, id, l_name, nationality, ph_no, salary, ssn];;
'Project [concat(f_name#447, ', , l_name#448) AS concat(f_name, , , l_name)#649]
+- LogicalRDD [id#446, f_name#447, l_name#448, salary#449, nationality#450, ph_no#451, ssn#452], false


In [65]:
employeeDF.select(concat(col("f_name"), lit(", "), col("l_name")).alias('full_name')).show()

+------------+
|   full_name|
+------------+
|Scott, Tiger|
| Henry, Ford|
+------------+



### String manipulation functions - 1<a id="fifth"></a>

Let us understand how to concatenate strings using concat function.

* Concatenating strings
    * We can pass a variable number of strings to `concat` function.
    * It will return one string concatenating all the strings.
    * If we have to concatenate literal in between then we have to use lit function.

* Case Conversion and Length
    * Convert all the alphabetic characters in a string to uppercase - `upper`
    * Convert all the alphabetic characters in a string to lowercase - `lower`
    * Convert first character in a string to uppercase - `initcap`
    * Get number of characters in a string - `length`
    * All the 4 functions take column type argument.

#### Concatenation
Let us perform few tasks to understand more about concat function.

* Let’s create a Data Frame and explore concat function.

In [79]:
employees = [(1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]

In [80]:
employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )

In [81]:
employeesDF.show()

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+--------------+----------------+-----------+



* Create a new column by name `full_name` concatenating `first_name` and `last_name`.


In [70]:
employeesDF. \
    withColumn("full_name", concat("first_name", "last_name")). \
    show()

+-----------+----------+---------+------+--------------+----------------+-----------+----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn| full_name|
+-----------+----------+---------+------+--------------+----------------+-----------+----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|ScottTiger|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123| HenryFord|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|NickJunior|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118| BillGomes|
+-----------+----------+---------+------+--------------+----------------+-----------+----------+



* Improvise by adding a **comma followed by a space** in between **first_name** and **last_name**.


In [72]:
employeesDF. \
    withColumn("full_name", concat("first_name", lit(', '), "last_name")). \
    show()

+-----------+----------+---------+------+--------------+----------------+-----------+------------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|   full_name|
+-----------+----------+---------+------+--------------+----------------+-----------+------------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|Scott, Tiger|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123| Henry, Ford|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|Nick, Junior|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118| Bill, Gomes|
+-----------+----------+---------+------+--------------+----------------+-----------+------------+



#### Case Conversion and Length

* Use employees data and create a Data Frame.
* Apply all 4 functions on **nationality** and see the results.

In [83]:
employeesDF.select('employee_id', 'nationality') \
.withColumn("nationality_upper", upper(col('nationality'))) \
.withColumn("nationality_lower", lower(col('nationality'))) \
.withColumn("nationality_initcap", initcap(col('nationality'))) \
.withColumn("nationality_length", length(col('nationality'))) \
.show()

+-----------+--------------+-----------------+-----------------+-------------------+------------------+
|employee_id|   nationality|nationality_upper|nationality_lower|nationality_initcap|nationality_length|
+-----------+--------------+-----------------+-----------------+-------------------+------------------+
|          1| united states|    UNITED STATES|    united states|      United States|                13|
|          2|         India|            INDIA|            india|              India|                 5|
|          3|united KINGDOM|   UNITED KINGDOM|   united kingdom|     United Kingdom|                14|
|          4|     AUSTRALIA|        AUSTRALIA|        australia|          Australia|                 9|
+-----------+--------------+-----------------+-----------------+-------------------+------------------+



####  Substring

Let us understand how we can extract substrings using function  `substring`
* If we are processing **fixed length columns** then we use `substring` to extract the information.
* Here are some of the examples for **fixed length columns** and the use cases for which we typically extract information.
* 9 Digit Social Security Number. We typically extract last 4 digits and provide it to the tele verification applications.
* 16 Digit Credit Card Number. We typically use first 4 digit number to identify Credit Card Provider and last 4 digits for the purpose of tele verification.
* Data coming from MainFrames systems are quite often fixed length. We might have to extract the information and store in multiple columns.
* `substring` function takes 3 arguments, **column**, **position**, **length**. We can also provide position from the end by passing negative value.

In [85]:
help(substring)

Help on function substring in module pyspark.sql.functions:

substring(str, pos, len)
    Substring starts at `pos` and is of length `len` when str is String type or
    returns the slice of byte array that starts at `pos` in byte and is of length `len`
    when str is Binary type.
    
    .. note:: The position is not zero based, but 1 based index.
    
    >>> df = spark.createDataFrame([('abcd',)], ['s',])
    >>> df.select(substring(df.s, 1, 2).alias('s')).collect()
    [Row(s='ab')]
    
    .. versionadded:: 1.5



In [84]:
l = [('X', )]
df = spark.createDataFrame(l, "dummy STRING")
df.show()

+-----+
|dummy|
+-----+
|    X|
+-----+



In [88]:
df.select(substring(lit("Hello World"), 7, 5)).show()

+----------------------------+
|substring(Hello World, 7, 5)|
+----------------------------+
|                       World|
+----------------------------+



In [89]:
df.select(substring(lit("Hello World"), -5, 5)).show()

+-----------------------------+
|substring(Hello World, -5, 5)|
+-----------------------------+
|                        World|
+-----------------------------+



#### Tasks - substring

Let us perform few tasks to extract information from fixed length strings.
* Create a list for employees with name, ssn and phone_number.
* SSN Format **3 2 4** - Fixed Length with 9 digits
* Phone Number Format - Country Code is variable and remaining phone number have 10 digits:
 * Country Code - one to 3 digits
 * Area Code - 3 digits
 * Phone Number Prefix - 3 digits
 * Phone Number Remaining - 4 digits
 * All the 4 parts are separated by spaces
* Create a Dataframe with column names name, ssn and phone_number
* Extract last 4 digits from the phone number.
* Extract last 4 digits from SSN.

In [93]:
employeesDF.select('employee_id', 'phone_number', 'ssn'). \
            withColumn('phone_last4', substring(col('phone_number'), -4, 4).cast("int")). \
            withColumn('ssn_last4', substring(col('ssn'), -4, 4).cast("int")). \
            show()

+-----------+----------------+-----------+-----------+---------+
|employee_id|    phone_number|        ssn|phone_last4|ssn_last4|
+-----------+----------------+-----------+-----------+---------+
|          1| +1 123 456 7890|123 45 6789|       7890|     6789|
|          2|+91 234 567 8901|456 78 9123|       8901|     9123|
|          3|+44 111 111 1111|222 33 4444|       1111|     4444|
|          4|+61 987 654 3210|789 12 6118|       3210|     6118|
+-----------+----------------+-----------+-----------+---------+



In [94]:
employeesDF.select('employee_id', 'phone_number', 'ssn'). \
            withColumn('phone_last4', substring(col('phone_number'), -4, 4).cast("int")). \
            withColumn('ssn_last4', substring(col('ssn'), -4, 4).cast("int")). \
            printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- ssn: string (nullable = true)
 |-- phone_last4: integer (nullable = true)
 |-- ssn_last4: integer (nullable = true)



In [90]:
employeesDF.show(5)

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+--------------+----------------+-----------+



#### Extracting Strings using split
Let us understand how we can extract substrings using  `split`.

* If we are processing **variable length columns** with **delimiter** then we use `split` to extract the information.
* Here are some of the examples for **variable length columns** and the use cases for which we typically extract information.
* Address where we store House Number, Street Name, City, State and Zip Code comma separated. We might want to extract City and State for demographics reports.
* `split` takes 2 arguments, **column** and **delimiter**.
* `split` convert each string into array and we can access the elements using index.

In [6]:
l = [('X',)]
df = spark.createDataFrame(l, 'dummy STRING')

In [7]:
df.select(split(lit("Hello World, how are you"), " ")).show(truncate=False)

+--------------------------------------+
|split(Hello World, how are you,  , -1)|
+--------------------------------------+
|[Hello, World,, how, are, you]        |
+--------------------------------------+



In [9]:
df.select(split(lit("Hello World, how are you"), " ")[2]).show(truncate=False)

+-----------------------------------------+
|split(Hello World, how are you,  , -1)[2]|
+-----------------------------------------+
|how                                      |
+-----------------------------------------+



In [11]:
df.select(split(lit("Hello World, how are you"), " ").alias('word')).show(truncate=False)

+------------------------------+
|word                          |
+------------------------------+
|[Hello, World,, how, are, you]|
+------------------------------+



* Most of the problems can be solved either by using `substring` or `split`.

#### Tasks
Let us perform few tasks to extract information from fixed length strings as well as delimited variable length strings.

* Create a list for employees with name, ssn and phone_number.
* SSN Format **3 2 4** - Fixed Length with 9 digits
* Phone Number Format - Country Code is variable and remaining phone number have 10 digits:
 * Country Code - one to 3 digits
 * Area Code - 3 digits
 * Phone Number Prefix - 3 digits
 * Phone Number Remaining - 4 digits
 * All the 4 parts are separated by spaces
* Create a Dataframe with column names name, ssn and phone_number
* Extract area code and last 4 digits from the phone number.
* Extract last 4 digits from SSN.

In [22]:
employees = [(1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111, +44 111 111 1123", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210, +61 987 654 3223", "789 12 6118"
                     )
                ]

In [30]:
employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, nationality STRING,
                    phone_numbers STRING, ssn STRING"""
                   )

In [31]:
employeesDF.select("employee_id", "phone_numbers") \
            .show()

+-----------+--------------------+
|employee_id|       phone_numbers|
+-----------+--------------------+
|          1|     +1 123 456 7890|
|          2|    +91 234 567 8901|
|          3|+44 111 111 1111,...|
|          4|+61 987 654 3210,...|
+-----------+--------------------+



In [32]:
employeesDF.select("employee_id", "phone_numbers", "ssn") \
            .withColumn("phone_number", explode(split(col("phone_numbers"), ", "))).show()

+-----------+--------------------+-----------+----------------+
|employee_id|       phone_numbers|        ssn|    phone_number|
+-----------+--------------------+-----------+----------------+
|          1|     +1 123 456 7890|123 45 6789| +1 123 456 7890|
|          2|    +91 234 567 8901|456 78 9123|+91 234 567 8901|
|          3|+44 111 111 1111,...|222 33 4444|+44 111 111 1111|
|          3|+44 111 111 1111,...|222 33 4444|+44 111 111 1123|
|          4|+61 987 654 3210,...|789 12 6118|+61 987 654 3210|
|          4|+61 987 654 3210,...|789 12 6118|+61 987 654 3223|
+-----------+--------------------+-----------+----------------+



In [38]:
employeesDF = employeesDF.select("employee_id", "phone_numbers", "ssn") \
            .withColumn("phone_number", explode(split(col("phone_numbers"), ", "))) 

In [39]:
employeesDF.show()

+-----------+--------------------+-----------+----------------+
|employee_id|       phone_numbers|        ssn|    phone_number|
+-----------+--------------------+-----------+----------------+
|          1|     +1 123 456 7890|123 45 6789| +1 123 456 7890|
|          2|    +91 234 567 8901|456 78 9123|+91 234 567 8901|
|          3|+44 111 111 1111,...|222 33 4444|+44 111 111 1111|
|          3|+44 111 111 1111,...|222 33 4444|+44 111 111 1123|
|          3|+44 111 111 1111,...|222 33 4444|+44 111 111 1111|
|          3|+44 111 111 1111,...|222 33 4444|+44 111 111 1123|
|          3|+44 111 111 1111,...|222 33 4444|+44 111 111 1111|
|          3|+44 111 111 1111,...|222 33 4444|+44 111 111 1123|
|          3|+44 111 111 1111,...|222 33 4444|+44 111 111 1111|
|          3|+44 111 111 1111,...|222 33 4444|+44 111 111 1123|
|          4|+61 987 654 3210,...|789 12 6118|+61 987 654 3210|
|          4|+61 987 654 3210,...|789 12 6118|+61 987 654 3223|
|          4|+61 987 654 3210,...|789 12

In [41]:
employeesDF.select("employee_id", "ssn", "phone_number") \
            .withColumn("area_code", split(col("phone_number"), " ")[1].cast("int")) \
            .withColumn("last4_phone_number", split(col("phone_number"), " ")[3].cast("int")) \
            .withColumn("last4_ssn", split(col("ssn"), " ")[2].cast("int")) \
            .show()

+-----------+-----------+----------------+---------+------------------+---------+
|employee_id|        ssn|    phone_number|area_code|last4_phone_number|last4_ssn|
+-----------+-----------+----------------+---------+------------------+---------+
|          1|123 45 6789| +1 123 456 7890|      123|              7890|     6789|
|          2|456 78 9123|+91 234 567 8901|      234|              8901|     9123|
|          3|222 33 4444|+44 111 111 1111|      111|              1111|     4444|
|          3|222 33 4444|+44 111 111 1123|      111|              1123|     4444|
|          3|222 33 4444|+44 111 111 1111|      111|              1111|     4444|
|          3|222 33 4444|+44 111 111 1123|      111|              1123|     4444|
|          3|222 33 4444|+44 111 111 1111|      111|              1111|     4444|
|          3|222 33 4444|+44 111 111 1123|      111|              1123|     4444|
|          3|222 33 4444|+44 111 111 1111|      111|              1111|     4444|
|          3|222

In [42]:
employeesDF \
    .groupBy('employee_id') \
    .count() \
    .show()

+-----------+-----+
|employee_id|count|
+-----------+-----+
|          1|    1|
|          3|    8|
|          4|    8|
|          2|    1|
+-----------+-----+



### String manipulation functions - 2<a id="sixth"></a>

#### String Manipulation - Padding
Let us understand how to pad characters at the beginning or at the end of strings.
* We typically pad characters to build fixed length values or records.
* Fixed length values or records are extensively used in Mainframes based systems.
* Length of each and every field in fixed length records is predetermined and if the value of the field is less than the predetermined length then we pad with a standard character.
* In terms of numeric fields we pad with zero on the leading or left side. For non numeric fields, we pad with some standard character on leading or trailing side.
* We use `lpad` to pad a string with a specific character on leading or left side and `rpad` to pad on trailing or right side.
* Both lpad and rpad, take 3 arguments - column or expression, desired length and the character need to be padded.

#### Tasks

Let us perform simple tasks to understand the syntax of `lpad` or `rpad`.
* Create a Dataframe with single value and single column.
* Apply `lpad` to pad with - to Hello to make it 10 characters.

In [3]:
l = [('X',)]
df = spark.createDataFrame(l, 'dummy STRING')
df.show()

+-----+
|dummy|
+-----+
|    X|
+-----+



In [5]:
df.select(lpad(lit('hello'), 10, '-').alias('dummy')).show()

+----------+
|     dummy|
+----------+
|-----hello|
+----------+



* Let’s take the **employees** Dataframe

In [6]:
employees = [(1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]

In [7]:
employeesDF = spark.createDataFrame(employees). \
    toDF("employee_id", "first_name",
         "last_name", "salary",
         "nationality", "phone_number",
         "ssn"
        )

In [10]:
employeesDF.printSchema()

root
 |-- employee_id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- nationality: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- ssn: string (nullable = true)



* Use **pad** functions to convert each of the field into fixed length and concatenate. Here are the details for each of the fields.
 * Length of the employee_id should be 5 characters and should be padded with zero.
 * Length of first_name and last_name should be 10 characters and should be padded with - on the right side.
 * Length of salary should be 10 characters and should be padded with zero.
 * Length of the nationality should be 15 characters and should be padded with - on the right side.
 * Length of the phone_number should be 17 characters and should be padded with - on the right side.
 * Length of the ssn can be left as is. It is 11 characters.
 
* Create a new Dataframe **empFixedDF** with column name **employee**. Preview the data by disabling truncate.

In [11]:
employeesDF.select(
                concat(
                    lpad(col('employee_id'), 5, '0'),
                    rpad(col('first_name'), 10, '-'),
                    rpad(col('last_name'), 10, '-'),
                    lpad(col('salary'), 5, '0'),
                    rpad(col('nationality'), 15, '-'),
                    rpad(col('phone_number'), 17, '-'),
                    'ssn').alias('employee')
            ).show(truncate=False)

+-------------------------------------------------------------------------+
|employee                                                                 |
+-------------------------------------------------------------------------+
|00001Scott-----Tiger-----1000.united states--+1 123 456 7890--123 45 6789|
|00002Henry-----Ford------1250.India----------+91 234 567 8901-456 78 9123|
|00003Nick------Junior----750.0united KINGDOM-+44 111 111 1111-222 33 4444|
|00004Bill------Gomes-----1500.AUSTRALIA------+61 987 654 3210-789 12 6118|
+-------------------------------------------------------------------------+



#### String Manipulation - Trimming
Let us understand how to trim unwanted leading and trailing characters around a string.
* We typically use trimming to remove unnecessary characters from fixed length records.
* Fixed length records are extensively used in Mainframes and we might have to process it using Spark.
* As part of processing we might want to remove leading or trailing characters such as 0 in case of numeric types and space or some standard character in case of alphanumeric types.
* As of now Spark trim functions take the column as argument and remove leading or trailing spaces.
* Trim spaces towards left - `ltrim`
* Trim spaces towards right - `rtrim`
* Trim spaces on both sides - `trim`

In [13]:
l = [('   Hello,     ',)]
df = spark.createDataFrame(l, 'dummy STRING')
df.show()

+--------------+
|         dummy|
+--------------+
|   Hello,     |
+--------------+



In [15]:
# Pyspark trim only remove spaces while sql style trim functions can trim any character
df.withColumn('ltrim', ltrim(col('dummy'))) \
    .withColumn('rtrim', rtrim(col('dummy'))) \
    .withColumn('trim', trim(col('dummy'))) \
    .show()

+--------------+-----------+---------+------+
|         dummy|      ltrim|    rtrim|  trim|
+--------------+-----------+---------+------+
|   Hello,     |Hello,     |   Hello,|Hello,|
+--------------+-----------+---------+------+



In [25]:
# For using sql style trim functions, use expr  
df.withColumn('ltrim', expr("ltrim(dummy)")) \
    .withColumn('rtrim', expr("rtrim(',', rtrim(dummy))")) \
    .withColumn('trim', expr("trim(dummy)")) \
    .show()

+--------------+-----------+--------+------+
|         dummy|      ltrim|   rtrim|  trim|
+--------------+-----------+--------+------+
|   Hello,     |Hello,     |   Hello|Hello,|
+--------------+-----------+--------+------+



In [27]:
spark.sql("DESCRIBE FUNCTION rtrim").show(truncate=False)

+-----------------------------------------------------------------------------+
|function_desc                                                                |
+-----------------------------------------------------------------------------+
|Function: rtrim                                                              |
|Class: org.apache.spark.sql.catalyst.expressions.StringTrimRight             |
|Usage: 
    rtrim(str) - Removes the trailing space characters from `str`.
  |
+-----------------------------------------------------------------------------+



In [28]:
spark.sql("DESCRIBE FUNCTION trim").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|function_desc                                                                                                                                                                                                                                                                             

In [31]:
df.withColumn('ltrim', expr("trim(LEADING FROM dummy)")) \
    .withColumn('rtrim', expr("trim(TRAILING ',' FROM rtrim(dummy))")) \
    .withColumn('trim', expr("trim(BOTH FROM dummy)")) \
    .show()

+--------------+-----------+--------+------+
|         dummy|      ltrim|   rtrim|  trim|
+--------------+-----------+--------+------+
|   Hello,     |Hello,     |   Hello|Hello,|
+--------------+-----------+--------+------+



### Date and Time overview<a id="seventh"></a>

* We can use `current_date` to get today’s server date. 
 * Date will be returned using **yyyy-MM-dd** format.
* We can use `current_timestamp` to get current server time. 
 * Timestamp will be returned using **yyyy-MM-dd HH:mm:ss:SSS** format.
 * Hours will be by default in 24 hour format.

In [32]:
l = [('X',)]
df = spark.createDataFrame(l, 'dummy STRING')
df.show()

+-----+
|dummy|
+-----+
|    X|
+-----+



In [33]:
df.select(current_date()).show()

+--------------+
|current_date()|
+--------------+
|    2022-01-29|
+--------------+



In [35]:
df.select(current_timestamp()).show(truncate=False)

+--------------------------+
|current_timestamp()       |
+--------------------------+
|2022-01-29 11:45:49.100927|
+--------------------------+



In [44]:
df.select(to_date(lit('20211201'), 'yyyyMMdd').alias('date')).show()

+----------+
|      date|
+----------+
|2021-12-01|
+----------+



In [37]:
# DDD represent day of the year
df.select(to_date(lit('2021021'), 'yyyyDDD').alias('date')).show()

+----------+
|      date|
+----------+
|2021-01-21|
+----------+



In [38]:
# MMM represent short form of month in string format
df.select(to_date(lit('02-Mar-2021'), 'dd-MMM-yyyy').alias('date')).show()

+----------+
|      date|
+----------+
|2021-03-02|
+----------+



In [39]:
# MMMM represent full form of month in string format
df.select(to_date(lit('02-March-2021'), 'dd-MMMM-yyyy').alias('date')).show()

+----------+
|      date|
+----------+
|2021-03-02|
+----------+



In [48]:
df.select(to_timestamp(lit('20211201 1725'), 'yyyyMMdd HHmm').alias('timestamp')).show()

+-------------------+
|          timestamp|
+-------------------+
|2021-12-01 17:25:00|
+-------------------+



### Date and Time arithmetic<a id="eigth"></a>

Let us perform Date and Time Arithmetic using relevant functions.
* Adding days to a date or timestamp - `date_add`
* Subtracting days from a date or timestamp - `date_sub`
* Getting difference between 2 dates or timestamps - `datediff`
* Getting a number of months between 2 dates or timestamps - `months_between`
* Adding months to a date or timestamp - `add_months`
* Getting next day from a given date - `next_day`
* All the functions are self explanatory. We can apply these on standard date or timestamp. All the functions return date even when applied on timestamp field.

#### Tasks

Let us perform some tasks related to date arithmetic.
* Get help on each and every function first and understand what all arguments need to be passed.
* Create a Dataframe by name datetimesDF with columns date and time.

In [3]:
datetimes = [("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                ]

In [4]:
datetimesDF = spark.createDataFrame(datetimes, "date STRING, time STRING")
datetimesDF.show(truncate=False)

+----------+-----------------------+
|date      |time                   |
+----------+-----------------------+
|2014-02-28|2014-02-28 10:00:00.123|
|2016-02-29|2016-02-29 08:08:08.999|
|2017-10-31|2017-12-31 11:59:59.123|
|2019-11-30|2019-08-31 00:00:00.000|
+----------+-----------------------+



* Add 10 days to both date and time values.
* Subtract 10 days from both date and time values.

In [7]:
help(date_add)

Help on function date_add in module pyspark.sql.functions:

date_add(start, days)
    Returns the date that is `days` days after `start`
    
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(date_add(df.dt, 1).alias('next_date')).collect()
    [Row(next_date=datetime.date(2015, 4, 9))]
    
    .. versionadded:: 1.5



In [13]:
datetimesDF.withColumn('date_add_date', date_add(col('date'), 10)) \
    .withColumn('date_add_time', date_sub(col('time'), 10)) \
    .withColumn('date_sub_date', date_sub(col('date'), 10)) \
    .withColumn('date_sub_time', date_sub(col('time'), 10)) \
    .show()

+----------+--------------------+-------------+-------------+-------------+-------------+
|      date|                time|date_add_date|date_add_time|date_sub_date|date_sub_time|
+----------+--------------------+-------------+-------------+-------------+-------------+
|2014-02-28|2014-02-28 10:00:...|   2014-03-10|   2014-02-18|   2014-02-18|   2014-02-18|
|2016-02-29|2016-02-29 08:08:...|   2016-03-10|   2016-02-19|   2016-02-19|   2016-02-19|
|2017-10-31|2017-12-31 11:59:...|   2017-11-10|   2017-12-21|   2017-10-21|   2017-12-21|
|2019-11-30|2019-08-31 00:00:...|   2019-12-10|   2019-08-21|   2019-11-20|   2019-08-21|
+----------+--------------------+-------------+-------------+-------------+-------------+



In [14]:
# If you pass negative number to date_add, it will subtract
# If you pass negative number to date_sub, it will add
datetimesDF.withColumn('date_add', date_add(col('date'), -10)) \
        .withColumn('date_sub', date_sub(col('date'), -10)).show()

+----------+--------------------+----------+----------+
|      date|                time|  date_add|  date_sub|
+----------+--------------------+----------+----------+
|2014-02-28|2014-02-28 10:00:...|2014-02-18|2014-03-10|
|2016-02-29|2016-02-29 08:08:...|2016-02-19|2016-03-10|
|2017-10-31|2017-12-31 11:59:...|2017-10-21|2017-11-10|
|2019-11-30|2019-08-31 00:00:...|2019-11-20|2019-12-10|
+----------+--------------------+----------+----------+



* Get the difference between current_date and date values as well as current_timestamp and time values.

In [16]:
help(datediff)

Help on function datediff in module pyspark.sql.functions:

datediff(end, start)
    Returns the number of days from `start` to `end`.
    
    >>> df = spark.createDataFrame([('2015-04-08','2015-05-10')], ['d1', 'd2'])
    >>> df.select(datediff(df.d2, df.d1).alias('diff')).collect()
    [Row(diff=32)]
    
    .. versionadded:: 1.5



In [15]:
datetimesDF \
    .withColumn("datediff_date", datediff(current_date(), col('date'))) \
    .withColumn("datediff_time", datediff(current_timestamp(), col('time'))) \
    .show()

+----------+--------------------+-------------+-------------+
|      date|                time|datediff_date|datediff_time|
+----------+--------------------+-------------+-------------+
|2014-02-28|2014-02-28 10:00:...|         2892|         2892|
|2016-02-29|2016-02-29 08:08:...|         2161|         2161|
|2017-10-31|2017-12-31 11:59:...|         1551|         1490|
|2019-11-30|2019-08-31 00:00:...|          791|          882|
+----------+--------------------+-------------+-------------+



* Get the number of months between current_date and date values as well as current_timestamp and time values.
* Add 3 months to both date values as well as time values.

In [22]:
datetimesDF \
    .withColumn("months_between_dates", round(months_between(current_date(), 'date'), 2)) \
    .withColumn("months_between_time", round(months_between(current_timestamp(), 'time'), 2)) \
    .withColumn("add_months_date", add_months('date', 3)) \
    .withColumn("add_months_time", add_months('time', 3)) \
    .show()

+----------+--------------------+--------------------+-------------------+---------------+---------------+
|      date|                time|months_between_dates|months_between_time|add_months_date|add_months_time|
+----------+--------------------+--------------------+-------------------+---------------+---------------+
|2014-02-28|2014-02-28 10:00:...|               95.03|              95.04|     2014-05-28|     2014-05-28|
|2016-02-29|2016-02-29 08:08:...|                71.0|               71.0|     2016-05-29|     2016-05-29|
|2017-10-31|2017-12-31 11:59:...|               50.94|              48.94|     2018-01-31|     2018-03-31|
|2019-11-30|2019-08-31 00:00:...|               25.97|              28.96|     2020-02-29|     2019-11-30|
+----------+--------------------+--------------------+-------------------+---------------+---------------+



### Date and Time - trunc and date_trunc<a id="ninth"></a>

In Data Warehousing we quite often run to date reports such as week to date, month to date, year to date etc.
* We can use `trunc` or `date_trunc` for the same to get the beginning date of the week, month, current year etc by passing date or timestamp to it.
* We can use `trunc` to get beginning date of the month or year by passing date or timestamp to it - for example `trunc(current_date(), "MM")` will give the first of the current month.
* We can use `date_trunc` to get beginning date of the month or year as well as beginning time of the day or hour by passing timestamp to it.
 * Get beginning date based on month - `date_trunc("MM", current_timestamp())`
 * Get beginning time based on day - `date_trunc("DAY", current_timestamp())`

#### Tasks

* Create a Dataframe by name datetimesDF with columns date and time.

In [24]:
datetimes = [("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                ]
datetimesDF = spark.createDataFrame(datetimes, schema="date STRING, time STRING")
datetimesDF.show(truncate=False)

+----------+-----------------------+
|date      |time                   |
+----------+-----------------------+
|2014-02-28|2014-02-28 10:00:00.123|
|2016-02-29|2016-02-29 08:08:08.999|
|2017-10-31|2017-12-31 11:59:59.123|
|2019-11-30|2019-08-31 00:00:00.000|
+----------+-----------------------+



* Get beginning month date using date field and beginning year date using time field.

In [28]:
help(trunc)

Help on function trunc in module pyspark.sql.functions:

trunc(date, format)
    Returns date truncated to the unit specified by the format.
    
    :param format: 'year', 'yyyy', 'yy' or 'month', 'mon', 'mm'
    
    >>> df = spark.createDataFrame([('1997-02-28',)], ['d'])
    >>> df.select(trunc(df.d, 'year').alias('year')).collect()
    [Row(year=datetime.date(1997, 1, 1))]
    >>> df.select(trunc(df.d, 'mon').alias('month')).collect()
    [Row(month=datetime.date(1997, 2, 1))]
    
    .. versionadded:: 1.5



In [29]:
help(date_trunc)

Help on function date_trunc in module pyspark.sql.functions:

date_trunc(format, timestamp)
    Returns timestamp truncated to the unit specified by the format.
    
    :param format: 'year', 'yyyy', 'yy', 'month', 'mon', 'mm',
        'day', 'dd', 'hour', 'minute', 'second', 'week', 'quarter'
    
    >>> df = spark.createDataFrame([('1997-02-28 05:02:11',)], ['t'])
    >>> df.select(date_trunc('year', df.t).alias('year')).collect()
    [Row(year=datetime.datetime(1997, 1, 1, 0, 0))]
    >>> df.select(date_trunc('mon', df.t).alias('month')).collect()
    [Row(month=datetime.datetime(1997, 2, 1, 0, 0))]
    
    .. versionadded:: 2.3



In [26]:
datetimesDF \
    .withColumn("datetrunc", trunc('date', 'MM')) \
    .withColumn("timetrunc", trunc('time', 'YY')).show(truncate=False)

+----------+-----------------------+----------+----------+
|date      |time                   |datetrunc |timetrunc |
+----------+-----------------------+----------+----------+
|2014-02-28|2014-02-28 10:00:00.123|2014-02-01|2014-01-01|
|2016-02-29|2016-02-29 08:08:08.999|2016-02-01|2016-01-01|
|2017-10-31|2017-12-31 11:59:59.123|2017-10-01|2017-01-01|
|2019-11-30|2019-08-31 00:00:00.000|2019-11-01|2019-01-01|
+----------+-----------------------+----------+----------+



* Get beginning hour time using date and time field.

In [27]:
datetimesDF \
    .withColumn("datetrunc", date_trunc('HOUR', 'date')) \
    .withColumn("timetrunc", date_trunc('HOUR', 'time')).show(truncate=False)

+----------+-----------------------+-------------------+-------------------+
|date      |time                   |datetrunc          |timetrunc          |
+----------+-----------------------+-------------------+-------------------+
|2014-02-28|2014-02-28 10:00:00.123|2014-02-28 00:00:00|2014-02-28 10:00:00|
|2016-02-29|2016-02-29 08:08:08.999|2016-02-29 00:00:00|2016-02-29 08:00:00|
|2017-10-31|2017-12-31 11:59:59.123|2017-10-31 00:00:00|2017-12-31 11:00:00|
|2019-11-30|2019-08-31 00:00:00.000|2019-11-30 00:00:00|2019-08-31 00:00:00|
+----------+-----------------------+-------------------+-------------------+



### Date and Time - extracting information<a id="tenth"></a>

Overview about Date and Time extract functions. Here are the extract functions that are useful which are self explanatory.

* `year`
* `month`
* `weekofyear`
* `dayofyear`
* `dayofmonth`
* `dayofweek`
* `hour`
* `minute`
* `second`

In [30]:
l = [('X',)]
df = spark.createDataFrame(l, 'dummy STRING')
df.show()

+-----+
|dummy|
+-----+
|    X|
+-----+



In [33]:
df.select(
    current_date().alias('current_date'))  \
    .withColumn('year', year(current_date())) \
    .withColumn('month', month(current_date())) \
    .withColumn('weekofyear', weekofyear(current_date())) \
    .withColumn('dayofyear', dayofyear(current_date())) \
    .withColumn('dayofmonth', dayofmonth(current_date())) \
    .withColumn('dayofweek', dayofweek(current_date())) \
    .withColumn('hour', hour(current_date())) \
    .withColumn('minute', minute(current_date())) \
    .withColumn('second', second(current_date())) \
.show()

+------------+----+-----+----------+---------+----------+---------+----+------+------+
|current_date|year|month|weekofyear|dayofyear|dayofmonth|dayofweek|hour|minute|second|
+------------+----+-----+----------+---------+----------+---------+----+------+------+
|  2022-01-29|2022|    1|         4|       29|        29|        7|   0|     0|     0|
+------------+----+-----+----------+---------+----------+---------+----+------+------+



In [34]:
df.select(
    current_timestamp().alias('current_date'))  \
    .withColumn('year', year(current_timestamp())) \
    .withColumn('month', month(current_timestamp())) \
    .withColumn('weekofyear', weekofyear(current_timestamp())) \
    .withColumn('dayofyear', dayofyear(current_timestamp())) \
    .withColumn('dayofmonth', dayofmonth(current_timestamp())) \
    .withColumn('dayofweek', dayofweek(current_timestamp())) \
    .withColumn('hour', hour(current_timestamp())) \
    .withColumn('minute', minute(current_timestamp())) \
    .withColumn('second', second(current_timestamp())) \
.show()

+--------------------+----+-----+----------+---------+----------+---------+----+------+------+
|        current_date|year|month|weekofyear|dayofyear|dayofmonth|dayofweek|hour|minute|second|
+--------------------+----+-----+----------+---------+----------+---------+----+------+------+
|2022-01-29 19:08:...|2022|    1|         4|       29|        29|        7|  19|     8|    20|
+--------------------+----+-----+----------+---------+----------+---------+----+------+------+



### Dealing with UNIX timestamp<a id="eleventh"></a>

Let us understand how to deal with Unix Timestamp in Spark.

* It is an integer and started from January 1st 1970 Midnight UTC.
* Beginning time is also known as epoch and is incremented by 1 every second.
* We can convert Unix Timestamp to regular date or timestamp and vice versa.
* We can use `unix_timestamp` to convert regular date or timestamp to a unix timestamp value. For example `unix_timestamp(lit("2019-11-19 00:00:00"))`
* We can use `from_unixtime` to convert unix timestamp to regular date or timestamp. For example `from_unixtime(lit(1574101800))`
* We can also pass format to both the functions.

In [4]:
datetimes = [(20140228, "2014-02-28", "2014-02-28 10:00:00.123"),
                     (20160229, "2016-02-29", "2016-02-29 08:08:08.999"),
                     (20171031, "2017-10-31", "2017-12-31 11:59:59.123"),
                     (20191130, "2019-11-30", "2019-08-31 00:00:00.000")
                ]

In [11]:
datetimesDF = spark.createDataFrame(datetimes).toDF("dateid", "date", "time")
datetimesDF.show(truncate=False)

+--------+----------+-----------------------+
|dateid  |date      |time                   |
+--------+----------+-----------------------+
|20140228|2014-02-28|2014-02-28 10:00:00.123|
|20160229|2016-02-29|2016-02-29 08:08:08.999|
|20171031|2017-10-31|2017-12-31 11:59:59.123|
|20191130|2019-11-30|2019-08-31 00:00:00.000|
+--------+----------+-----------------------+



* Get unix timestamp for dateid, date and time.

In [6]:
datetimesDF.printSchema()

root
 |-- dateid: long (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)



In [10]:
help(unix_timestamp)

Help on function unix_timestamp in module pyspark.sql.functions:

unix_timestamp(timestamp=None, format='yyyy-MM-dd HH:mm:ss')
    Convert time string with given pattern ('yyyy-MM-dd HH:mm:ss', by default)
    to Unix time stamp (in seconds), using the default timezone and the default
    locale, return null if fail.
    
    if `timestamp` is None, then it returns current timestamp.
    
    >>> spark.conf.set("spark.sql.session.timeZone", "America/Los_Angeles")
    >>> time_df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> time_df.select(unix_timestamp('dt', 'yyyy-MM-dd').alias('unix_time')).collect()
    [Row(unix_time=1428476400)]
    >>> spark.conf.unset("spark.sql.session.timeZone")
    
    .. versionadded:: 1.5



In [12]:
# unix_timestamp accept string value
datetimesDF. \
withColumn("unix_date_id", unix_timestamp(col("dateid").cast("string"), 'yyyyMMdd')). \
withColumn("unix_date", unix_timestamp("date", "yyyy-MM-dd")). \
withColumn("unix_time", unix_timestamp("time", "yyyy-MM-dd HH:mm:ss.SSS")). \
show()

+--------+----------+--------------------+------------+----------+----------+
|  dateid|      date|                time|unix_date_id| unix_date| unix_time|
+--------+----------+--------------------+------------+----------+----------+
|20140228|2014-02-28|2014-02-28 10:00:...|  1393525800|1393525800|1393561800|
|20160229|2016-02-29|2016-02-29 08:08:...|  1456684200|1456684200|1456713488|
|20171031|2017-10-31|2017-12-31 11:59:...|  1509388200|1509388200|1514701799|
|20191130|2019-11-30|2019-08-31 00:00:...|  1575052200|1575052200|1567189800|
+--------+----------+--------------------+------------+----------+----------+



* Create a dataframe by name unixtimesDF with one column unixtime using 4 values. You can use the unix timestamp generated for time column in previous task.

In [14]:
# unix_timestamp accepts string of format yyyy-MM-dd HH:mm:ss by default
# If you do not pass the format, it will return null 
datetimesDF. \
withColumn("unix_date_id", unix_timestamp(col("dateid").cast("string"), 'yyyyMMdd')). \
withColumn("unix_date", unix_timestamp("date")). \
withColumn("unix_time", unix_timestamp("time", "yyyy-MM-dd HH:mm:ss.SSS")). \
show()

+--------+----------+--------------------+------------+---------+----------+
|  dateid|      date|                time|unix_date_id|unix_date| unix_time|
+--------+----------+--------------------+------------+---------+----------+
|20140228|2014-02-28|2014-02-28 10:00:...|  1393525800|     null|1393561800|
|20160229|2016-02-29|2016-02-29 08:08:...|  1456684200|     null|1456713488|
|20171031|2017-10-31|2017-12-31 11:59:...|  1509388200|     null|1514701799|
|20191130|2019-11-30|2019-08-31 00:00:...|  1575052200|     null|1567189800|
+--------+----------+--------------------+------------+---------+----------+



In [15]:
unixtimes = [
(1393561800, ),
(1456713488, ),
(1514701799, ),
(1567189800, )
]

In [16]:
unixtimesDF = spark.createDataFrame(unixtimes).toDF("unixtime")
unixtimesDF.show()

+----------+
|  unixtime|
+----------+
|1393561800|
|1456713488|
|1514701799|
|1567189800|
+----------+



In [17]:
unixtimesDF.printSchema()

root
 |-- unixtime: long (nullable = true)



In [21]:
unixtimesDF. \
withColumn("Date", from_unixtime("unixtime", "yyyyMMdd")). \
withColumn("time", from_unixtime("unixtime")) \
.show()

+----------+--------+-------------------+
|  unixtime|    Date|               time|
+----------+--------+-------------------+
|1393561800|20140228|2014-02-28 10:00:00|
|1456713488|20160229|2016-02-29 08:08:08|
|1514701799|20171231|2017-12-31 11:59:59|
|1567189800|20190831|2019-08-31 00:00:00|
+----------+--------+-------------------+



### Dealing with Nulls<a id="twelth"></a>

* We can use `coalesce` to return first non null value.
* We also have traditional SQL style functions such as `nvl`. However, they can be used either with `expr` or `selectExpr`.

In [1]:
employees = [(1, "Scott", "Tiger", 1000.0, 10,
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, None,
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, '',
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 10,
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]

In [5]:
employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, bonus STRING, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )

In [8]:
help(coalesce)

Help on function coalesce in module pyspark.sql.functions:

coalesce(*cols)
    Returns the first column that is not null.
    
    >>> cDf = spark.createDataFrame([(None, None), (1, None), (None, 2)], ("a", "b"))
    >>> cDf.show()
    +----+----+
    |   a|   b|
    +----+----+
    |null|null|
    |   1|null|
    |null|   2|
    +----+----+
    
    >>> cDf.select(coalesce(cDf["a"], cDf["b"])).show()
    +--------------+
    |coalesce(a, b)|
    +--------------+
    |          null|
    |             1|
    |             2|
    +--------------+
    
    >>> cDf.select('*', coalesce(cDf["a"], lit(0.0))).show()
    +----+----+----------------+
    |   a|   b|coalesce(a, 0.0)|
    +----+----+----------------+
    |null|null|             0.0|
    |   1|null|             1.0|
    |null|   2|             0.0|
    +----+----+----------------+
    
    .. versionadded:: 1.4



In [7]:
employeesDF. \
withColumn('nulls', coalesce('bonus', lit(0))). \
show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|nulls|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|   10|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|    0|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|     |
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|   10|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----+



In [9]:
# Convert the empty string to int, it will return null
employeesDF. \
withColumn('nulls', coalesce(col('bonus').cast('int'), lit(0))). \
show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|nulls|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|   10|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|    0|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|    0|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|   10|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----+



In [10]:
# Find total payment including salary and bonus
employeesDF. \
withColumn('payment', col('salary') + (coalesce(col('bonus').cast('int'), lit(0)) / 100)). \
show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+-------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|payment|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789| 1000.1|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123| 1250.0|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|  750.0|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118| 1500.1|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-------+



In [11]:
# SQL like statement for handling nulls or empties
employeesDF. \
withColumn('nulls', expr("nvl(bonus, 0)")). \
show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|nulls|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|   10|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|    0|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|     |
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|   10|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----+



In [12]:
# Replace null with empty string then fill 0
employeesDF. \
withColumn('nulls', expr("nvl(nullif(bonus, ''), 0)")). \
show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|nulls|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|   10|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|    0|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|    0|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|   10|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----+



### Conclusion<a id="thirteen"></a>

### Using CASE and WHEN

* `CASE` and `WHEN` is typically used to apply transformations based on conditions. We can use `CASE` and `WHEN` similar to SQL using `expr` or `selectExpr`.
* If we want to use APIs, Spark provides functions such as `when` and `otherwise`. `when` is available as part of `pyspark.sql.functions`. On top of column type that is generated using `when` we should be able to invoke `otherwise`.

In [16]:
employeesDF.withColumn("bonus", 
                      expr(""" 
                      CASE WHEN bonus is NULL OR bonus='' THEN 0 ELSE bonus end
                      """)).show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0|    0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|    0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+



In [18]:
when?

In [19]:
employeesDF.withColumn("bonus", 
                      when((col('bonus').isNull()) | (col('bonus') == lit('')), 0).otherwise(col('bonus'))).show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0|    0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|    0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+



* Create a dataframe using list called as persons and categorize them based up on following rules.

| Age Range | Category |
| ---       |      --- |
| 0 to 2 Months| New Born |
| 2+ to 12 Months| Infant |
| 12+ to 48 Months| Toddler |
| 48+ to 144 Months| Kids |
| 144+ Months| Teenager or Adult |


In [21]:
persons = [
    (1, 1),
    (2, 13),
    (3, 18),
    (4, 60),
    (5, 120),
    (6, 0),
    (7, 12),
    (8, 160)
]

In [22]:
personsDF = spark.createDataFrame(persons, schema='id INT, age INT')
personsDF.show()

+---+---+
| id|age|
+---+---+
|  1|  1|
|  2| 13|
|  3| 18|
|  4| 60|
|  5|120|
|  6|  0|
|  7| 12|
|  8|160|
+---+---+



In [24]:
personsDF. \
withColumn('category',
          expr("""
          CASE WHEN age BETWEEN 0 AND 2 THEN 'New Born'
               WHEN age > 2 AND age <= 12 THEN 'Infant'
               WHEN age > 12 AND age <= 48 THEN 'Toddler'
               WHEN age > 48 AND age <= 144 THEN 'Kid'
               ELSE 'Teenager or Adult'
               END
          """)).show()

+---+---+-----------------+
| id|age|         category|
+---+---+-----------------+
|  1|  1|         New Born|
|  2| 13|          Toddler|
|  3| 18|          Toddler|
|  4| 60|              Kid|
|  5|120|              Kid|
|  6|  0|         New Born|
|  7| 12|           Infant|
|  8|160|Teenager or Adult|
+---+---+-----------------+



In [28]:
personsDF. \
withColumn('category',
               when(col('age').between(0, 2), 'New Born').
               when((col('age') > 2) & (col('age') <= 12), 'Infant').
               when((col('age') > 12) & (col('age') <= 48), 'Toddler').
               when((col('age') > 48) & (col('age') <= 144), 'Kid').
               otherwise('Teenager or Adult')).show()

+---+---+-----------------+
| id|age|         category|
+---+---+-----------------+
|  1|  1|         New Born|
|  2| 13|          Toddler|
|  3| 18|          Toddler|
|  4| 60|              Kid|
|  5|120|              Kid|
|  6|  0|         New Born|
|  7| 12|           Infant|
|  8|160|Teenager or Adult|
+---+---+-----------------+

